In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import tensorflow as tf
from sklearn import svm,metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE 
import seaborn as sn


import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

def InputAssembleNormalize(df,colunas):
  X = df[colunas]
  for x in colunas:
    X[x]=(X[x]-X[x].min())/(X[x].max()-X[x].min())
  return X


df_crm1 = pd.read_csv(r'CRM_v6.csv',decimal=",")
df_cred = pd.read_csv(r'Credito_v6.csv',decimal=",")
df_imov = pd.read_csv(r'Imovel_v6.csv',decimal=",")

#Acertando a primary key e target 
#Forced cleaning wrangling

df_crm1 = df_crm1.rename(columns={'Data Hora Aprovação Comite': 'Aprovação_Comite'})
df_crm1 = df_crm1.rename(columns={'Nome': 'Proposta'})

df_crm1.loc[:,['Aprovação_Comite']] = (~df_crm1['Aprovação_Comite'].isna()).astype(int)

#Escolhe colunas que são importantes de cada arquivo



colunasCRM = ['Aprovação_Comite','Proposta','Status','Valor Solicitado',
              'Quantidade de Dívidas', 'Valor Total Dívidas Real',
              'Bacen Valor à Vencer','Quantidade de Imóveis']

colunasCred = ['Proposta','Score Serasa','Score Neurolake','Total Bens e Direitos']

colunasImov = ['Proposta','Valor Médio de Venda Forçada']
#'Valor Total Dívidas Real',

colunas = ['Score Neurolake','Score Serasa','Valor Solicitado',
           'Quantidade de Dívidas', 'Valor Total Dívidas Real', 'Valor Médio de Venda Forçada',
           'Bacen Valor à Vencer','Total Bens e Direitos']


df_crm1 = df_crm1[colunasCRM]
df_cred = df_cred[colunasCred]
df_imov = df_imov[colunasImov]

#transforma Rascunho em numerio para não sair do groupby

df_crm1.loc[:,['Status']] = (df_crm1['Status'] == 'Rascunho').astype(int)

#concatena arquivos CRM e Crédito

lista = [df_crm1,df_cred,df_imov]

df_crm2 = pd.concat(lista)

#Agrupa pela Proposta considerando a média de score com os proponentes

df_crm = df_crm2.groupby('Proposta').mean()

plot_size = plt.rcParams["figure.figsize"]
plot_size [0] = 10
plot_size [1] = 7
plt.rcParams["figure.figsize"] = plot_size

corrMatrix = df_crm.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import tensorflow as tf
from sklearn import svm,metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE 
import seaborn as sn


import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

def InputAssembleNormalize(df,colunas):
  X = df[colunas]
  for x in colunas:
    X[x]=(X[x]-X[x].min())/(X[x].max()-X[x].min())
  return X


df_crm1 = pd.read_csv(r'CRM_v6.csv',decimal=",")
df_cred = pd.read_csv(r'Credito_v6.csv',decimal=",")
df_imov = pd.read_csv(r'Imovel_v6.csv',decimal=",")


In [ ]:
df_crm1.describe().T.head(50)

In [0]:
df_crm1 = df_crm1.rename(columns={'Data Hora Aprovação Comite': 'Aprovação_Comite'})
df_crm1 = df_crm1.rename(columns={'Nome': 'Proposta'})

In [23]:
df_crm1['Aprovação_Comite'].isna().value_counts()

True     3555
False     226
Name: Aprovação_Comite, dtype: int64

In [69]:
df_crm1.loc[:,['Aprovação_Comite']] = (~df_crm1['Aprovação_Comite'].isna()).astype(int)

df_crm1['Aprovação_Comite'].value_counts()

0    3555
1     226
Name: Aprovação_Comite, dtype: int64

In [70]:
comp = df_crm1['Comprometimento de Renda']>100
apro = df_crm1['Aprovação_Comite']==1
print(len(df_crm1[comp & apro]))




0


In [0]:
df_crm1.drop(index=df_crm1[comp].index, inplace=True)

In [61]:
len(df_crm1)
df_crm1['Aprovação_Comite'].value_counts()

0    3501
1     226
Name: Aprovação_Comite, dtype: int64

In [73]:
df_crm1.Status.value_counts()

Rascunho    2564
Fechado      999
Ativo        164
Name: Status, dtype: int64

In [81]:
rasc = df_crm1['Status'] =='Rascunho'
descarta = (rasc & ~apro)
print(len(df_crm1[descarta]))
df_desc = df_crm1.drop(index=df_crm1[descarta].index)


2552


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [82]:
df_desc['Aprovação_Comite'].value_counts()

0    949
1    226
Name: Aprovação_Comite, dtype: int64

In [83]:
df_crm1.drop(index=df_crm1[descarta].index, inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [84]:
df_crm1['Aprovação_Comite'].value_counts()

0    949
1    226
Name: Aprovação_Comite, dtype: int64

In [0]:

colunasCRM = ['Aprovação_Comite','Proposta','Status','Valor Solicitado',
              'Quantidade de Dívidas', 'Valor Total Dívidas Real',
              'Bacen Valor à Vencer','Quantidade de Imóveis']

colunasCred = ['Proposta','Score Serasa','Score Neurolake','Total Bens e Direitos']

colunasImov = ['Proposta','Valor Médio de Venda Forçada']
#'Valor Total Dívidas Real',

colunas = ['Score Neurolake','Score Serasa','Valor Solicitado',
           'Quantidade de Dívidas', 'Valor Total Dívidas Real', 'Valor Médio de Venda Forçada',
           'Bacen Valor à Vencer','Total Bens e Direitos']


df_crm1 = df_crm1[colunasCRM]
df_cred = df_cred[colunasCred]
df_imov = df_imov[colunasImov]

#transforma Rascunho em numerio para não sair do groupby

df_crm1.loc[:,['Status']] = (df_crm1['Status'] == 'Rascunho').astype(int)

#concatena arquivos CRM e Crédito

lista = [df_crm1,df_cred,df_imov]

df_crm2 = pd.concat(lista)

In [88]:
df_crm2['Proposta'].value_counts().head(20)

PRO-03918-M9P1X1    38
PRO-03166-L0Y5W9    30
PRO-05152-X1M5X6    23
PRO-03757-Y6X2T3    21
PRO-05987-W6C5R5    18
PRO-02986-P9H6Q7    16
PRO-03628-G8B3F0    16
PRO-03382-V6L9D3    15
PRO-02577-P8G7L4    14
PRO-03198-Z9S0H7    14
PRO-02824-M3J0J4    14
PRO-02910-R3L1D7    13
PRO-03583-V4T2M2    12
PRO-02856-J1Z2X6    12
PRO-03039-B1V9V4    11
PRO-03996-X4T5N0    11
PRO-02057-L8D5S8    11
PRO-05643-B5V8R4    11
PRO-03422-T2G2F5    11
PRO-04796-S5X5W5    11
Name: Proposta, dtype: int64

In [0]:
df_crm = df_crm2.groupby('Proposta').mean()
df_crm.fillna(0,inplace=True)


In [94]:
df_crm

,Aprovação_Comite,Status,Valor Solicitado,Quantidade de Dívidas,Valor Total Dívidas Real,Bacen Valor à Vencer,Quantidade de Imóveis,Score Serasa,Score Neurolake,Total Bens e Direitos,Valor Médio de Venda Forçada
Proposta,,,,,,,,,,,
PRO-01406-X9S0J8,0.0,0.0,200000.0,0.0,0.0,0.00,1.0,0.0,37.710,0.00,736026.54
PRO-01407-J7H2Q8,1.0,0.0,150000.0,16.0,147760.8,2050054.76,1.0,0.0,43.730,359131.12,262162.50
PRO-01408-G1N8N4,0.0,0.0,200000.0,0.0,0.0,0.00,1.0,0.0,0.000,0.00,0.00
PRO-01409-H3Q3H3,0.0,0.0,50000.0,0.0,0.0,0.00,1.0,0.0,23.540,0.00,0.00
PRO-01410-N5P4X7,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,47.480,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
PRO-06231-S2W8V4,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,5.140,0.00,0.00
PRO-06232-V9B5Q1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,20.530,0.00,0.00
PRO-06233-R6C2H7,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,6.465,0.00,0.00
